In [1]:
import os
from openmm import Platform, LangevinMiddleIntegrator, XmlSerializer
import openmm.app as app
from openmm.unit import picosecond, kelvin
from openmm import unit
import parmed
import openmm
import MDAnalysis as md
from MDAnalysis.analysis import dihedrals
import nglview as ng
import numpy as np
import matplotlib.pyplot as plt
import sys
import parmed
import torch

import mymd

In [2]:
# mode = 'benzamidine'
# mode = 'ala15'
mode = '1ytc'

if mode == 'ala15':
    data_path = 'data/ala15/'
    pdb_file = os.path.join(data_path, 'ala15_new.pdb')
    prmtop_file = os.path.join(data_path, 'ala15.prmtop')
elif mode == '1ytc':
    data_path = 'data/1ytc/'
    pdb_file = os.path.join(data_path, '1ytc_fixed_mymd.pdb')
    prmtop_file = os.path.join(data_path, '1ytc_fixed_mymd.prmtop')
elif mode == 'benzamidine':
    data_path = 'data/benzamidine_torchmd/'
    pdb_file = os.path.join(data_path, 'structure.pdb')
    prmtop_file = os.path.join(data_path, 'structure.prmtop')

cutoff = 9
box = np.array([20., 20., 20.])
if mode == '1ytc':
    box = np.array([200., 200., 200.])
T = 300
device = 'cpu'
precision = torch.float
use_centered = True

In [3]:
mol = mymd.get_molecule(prmtop_file=prmtop_file, pdb_file=pdb_file)

2023-05-28 16:11:24,614 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.


In [4]:
class MyAmberParameterSet(parmed.amber.AmberParameterSet):
    """Inherited from AmberParameterSet to modify from_structure method."""

    @classmethod
    def my_from_structure(cls, struct):
        return super(parmed.amber.AmberParameterSet, cls).from_structure(struct, allow_unequal_duplicates=True)


structure = parmed.load_file(prmtop_file)
try:
    prm = parmed.amber.AmberParameterSet().from_structure(structure)
except:
    print('wrong')
    prm = MyAmberParameterSet.my_from_structure(structure)
print(isinstance(structure, parmed.amber.AmberParm))

a = unit.Quantity((box[0] * unit.angstrom, 0 * unit.angstrom, 0 * unit.angstrom))
b = unit.Quantity((0 * unit.angstrom, box[1] * unit.angstrom, 0 * unit.angstrom))
c = unit.Quantity((0 * unit.angstrom, 0 * unit.angstrom, box[2] * unit.angstrom))
structure.box_vectors = (a, b, c)


def disableDispersionCorrection(system):
    # According to openMM:
    # The long range dispersion correction is primarily useful when running simulations at constant pressure, since it
    # produces a more accurate variation in system energy with respect to volume.
    # So I will disable it to avoid implementing it for now.
    from openmm import NonbondedForce

    for f in system.getForces():
        if isinstance(f, NonbondedForce):
            f.setUseDispersionCorrection(False)


system = structure.createSystem(nonbondedMethod=app.CutoffPeriodic,
                                nonbondedCutoff=cutoff * unit.angstrom,
                                switchDistance=0)
system.setDefaultPeriodicBoxVectors(a, b, c)
disableDispersionCorrection(system)

integrator = openmm.LangevinIntegrator(T * unit.kelvin, 1 / unit.picoseconds, 2 * unit.femtoseconds)
platform = openmm.Platform.getPlatformByName('OpenCL')
properties = dict()

context = openmm.Context(system, integrator, platform, properties)
context.setPositions(mol.coords * unit.angstrom)
energies = parmed.openmm.energy_decomposition(structure, context)
energies

2023-05-28 16:11:24,920 - parmed.structure - INFO - Adding bonds...
2023-05-28 16:11:24,924 - parmed.structure - INFO - Adding angles...
2023-05-28 16:11:24,928 - parmed.structure - INFO - Adding dihedrals...
2023-05-28 16:11:24,938 - parmed.structure - INFO - Adding Ryckaert-Bellemans torsions...
2023-05-28 16:11:24,939 - parmed.structure - INFO - Adding Urey-Bradleys...
2023-05-28 16:11:24,939 - parmed.structure - INFO - Adding improper torsions...
2023-05-28 16:11:24,940 - parmed.structure - INFO - Adding CMAP torsions...
2023-05-28 16:11:24,940 - parmed.structure - INFO - Adding trigonal angle terms...
2023-05-28 16:11:24,940 - parmed.structure - INFO - Adding out-of-plane bends...
2023-05-28 16:11:24,941 - parmed.structure - INFO - Adding pi-torsions...
2023-05-28 16:11:24,941 - parmed.structure - INFO - Adding stretch-bends...
2023-05-28 16:11:24,942 - parmed.structure - INFO - Adding torsion-torsions...
2023-05-28 16:11:24,942 - parmed.structure - INFO - Adding Nonbonded force..

wrong
True


{'bond': 9562.422509858987,
 'angle': 4642.425945566443,
 'dihedral': 2021.953983926864,
 'nonbonded': 1602.1205690577199,
 'total': 17828.92417543021}

In [5]:
try:
    ff = mymd.PrmtopMolForceField(mol, prmtop_file, allow_unequal_duplicates=False)
except:
    print('False causes error, use True.')
    ff = mymd.PrmtopMolForceField(mol, prmtop_file, allow_unequal_duplicates=True)

system = mymd.System(mol, ff, cutoff=cutoff)
system.set_periodic_box_manual(box.reshape(3, 1))
integrator = mymd.VelocityVerletIntegrator(2)

simulation = mymd.Simulation(mol, system, integrator, device, precision, use_centered=use_centered)
simulation.set_positions(mol.coords)
simulation.set_velocities_to_temperature(T=T)
simulation.update_potentials_and_forces()

print(simulation.potentials)
print(simulation.potentials_sum)

False causes error, use True.
{'bonds': 9543.9296875, 'angles': 4640.02392578125, 'dihedrals': 2241.162109375, 'impropers': 68.10431671142578, 'lj': 288.13677978515625, 'electrostatics': -1567.444091796875, 'external': 0.0}
15213.912727355957
